<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/SolarPowerManagementSystemFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import metrics

In [110]:
class MyEnv3(gym.Env):
    
      def __init__(self,seed=42):
        np.random.seed(seed)
        self.action_space = Discrete(4)
        self.observation_space=MultiDiscrete([150,130,50])
        self.battery_cap=30
        self.max_battery_discharge=15
        self.max_battery_charge=10
        self.max_battery_capacity=50
        self.state,self.reward, self.done, self.info= None, None, None,None
        
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
      def _render(self):
        txt="state:"+str(self.state)+"reward"+str(self.reward)+"info:"+str(self.info)
        print(txt)

      def _states(self):
        observation_space=[]
        return observation_space
      def _reset(self):
        self.battery_cap=30
        self.state,self.reward,self.done,self.info=[30,0,0],0,False,{}
        return [30,0,0]

      def _step(self,action):
        done=False
        grid_export_=0
        grid_import_=0
        #load=np.random.randint(100,140)
        #pv=np.random.randint(80,130)
        battery_state=self.battery_cap
        reward=0
        data=[]
        if action== 0:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 1:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          # only battery charge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 2:
            grid_export_=0
            self.battery_cap= min(max(self.battery_cap-self.max_battery_discharge,0),50)# battery discharge
            grid_import_=max(load-pv-self.max_battery_discharge,0)
            info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
             }
        if action== 3:
          grid_export_=0
          self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)#change in battery
          grid_import_=max(load-pv-self.max_battery_discharge,0)
         #only battery discharge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
                    }     
        
        reward=grid_export_*8-grid_import_*10
        
        if(self.battery_cap<7.0):
              done=True
        #data.append([battery_state,load,pv,action,reward])
        #state.append(load)
        #state.append(pv)
        return reward,self.battery_cap, done,info

In [111]:
env=MyEnv3()

In [112]:
import time # Necessary to evaluate frugality
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [113]:
class DeepQL(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, nb_actions):
        super(DeepQL, self).__init__()
        self.input_dims = 3
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.nb_actions = 4
        self.fc1 = nn.Linear(self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.nb_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        
        if torch.cuda.is_available():       
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        self.to(self.device)
        
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        
        return actions

In [114]:
class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, nb_actions,
                max_mem_size = 100000, eps_end = 0.01, eps_dec = 5e-4):
        
        self.gamma = gamma
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(nb_actions)]
        self.mem_size = max_mem_size
        self.mem_counter = 0
        
        self.Q_eval = DeepQL(self.lr, nb_actions=nb_actions, input_dims=input_dims,
                            fc1_dims=64, fc2_dims=64)
        
        self.state_memory = np.zeros((self.mem_size, input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, input_dims), dtype=np.float32)
        
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype= np.bool)
    
    def store_transition(self, state, action ,reward, state_, done):
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.new_state_memory[index] = state_
        self.terminal_memory[index] = done
        
        self.mem_counter += 1
    
    def choose_action (self, observation):
        if np.random.random() > self.epsilon:
            state = torch.FloatTensor([observation]).to(self.Q_eval.device)
            actions = self.Q_eval.forward(state)
            action = torch.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        
        return action
    def train(self):
        if self.mem_counter < self.batch_size:
            return
        
        self.Q_eval.optimizer.zero_grad()
        
        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem,self.batch_size, replace=False)
        
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        
        state_batch = torch.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = torch.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = torch.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = torch.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        
        action_batch = self.action_memory[batch]
        
        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0
        
        q_target = reward_batch + self.gamma * torch.max(q_next,dim=1)[0]
        
        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()
        
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min
    

In [115]:
agent = Agent(gamma=0.99, epsilon=1.0, batch_size=128, nb_actions=4,
             eps_end=0.01, input_dims=3, lr= 0.0001)

<ipython-input-114-7406a06ab070>:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.terminal_memory = np.zeros(self.mem_size, dtype= np.bool)


In [118]:
done=False
for e in range(100):
  score = 0
  #load=np.random.randint(100,140)
  #pv=np.random.randint(80,130)
  state=env._reset()
  done=False
  #state_=[]
  #state_.append(state[0])
  while not done:
    state_=[]
    state_.append(state[0])
    load=np.random.randint(100,140)
    pv=np.random.randint(80,130)
    state_.append(load)
    state_.append(pv)
    print(state_)
    #state1=state_
    #action = env.action_space.sample()
    action=agent.choose_action(state_)
    print(action)
    reward,observation2, done, info = env._step(action)
    score+=round(reward)
    print(observation2)
    print(info)
    state_[0]=observation2
    #agent.store_transition()
  print('episode:{} Profit:{}'.format(e,score))

[30, 101, 117]
1
40
{'grid_export': 6, 'grid_import': 0}
[30, 122, 109]
1
27
{'grid_export': 0, 'grid_import': 0}
[30, 135, 84]
2
12
{'grid_export': 0, 'grid_import': 36}
[30, 115, 121]
0
18
{'grid_export': 0, 'grid_import': 0}
[30, 115, 91]
3
3
{'grid_export': 0, 'grid_import': 9}
episode:0 Profit:-402
[30, 107, 99]
1
22
{'grid_export': 0, 'grid_import': 0}
[30, 122, 107]
1
7
{'grid_export': 0, 'grid_import': 0}
[30, 104, 99]
3
2
{'grid_export': 0, 'grid_import': 0}
episode:1 Profit:0
[30, 100, 109]
1
39
{'grid_export': 0, 'grid_import': 0}
[30, 121, 97]
3
24
{'grid_export': 0, 'grid_import': 9}
[30, 132, 95]
3
9
{'grid_export': 0, 'grid_import': 22}
[30, 133, 93]
2
0
{'grid_export': 0, 'grid_import': 25}
episode:2 Profit:-560
[30, 101, 120]
2
15
{'grid_export': 0, 'grid_import': 0}
[30, 108, 80]
3
0
{'grid_export': 0, 'grid_import': 13}
episode:3 Profit:-130
[30, 122, 105]
2
15
{'grid_export': 0, 'grid_import': 2}
[30, 105, 114]
2
0
{'grid_export': 0, 'grid_import': 0}
episode:4 Prof